## Get the movie list - out of the imdb website

In [1]:
import os
import numpy as np
import codecs
path_baptiste = "/home/baptiste/Documents/data/train"
path_igor = "C:/Users/Igor/Documents/Master Data Science/Big Data Analytics/Projet/Data/train"
path_sofia = "/Users/Flukmacdesof/data 2/train"



#assumes labelled data ra stored into a positive and negative folder
#returns two lists one with the text per file and another with the corresponding class 
def loadLabeled(path):

	rootdirPOS =path+'/pos'
	rootdirNEG =path+'/neg'
	data=[]
	Class=[]
	count=0
	for subdir, dirs, files in os.walk(rootdirPOS):
		
		for file in files:
			with codecs.open(rootdirPOS+"/"+file, 'r',encoding="utf-8") as content_file:
				content = content_file.read() #assume that there are NO "new line characters"
				data.append(content)
	tmpc1=np.ones(len(data))
	for subdir, dirs, files in os.walk(rootdirNEG):
		
		for file in files:
			with codecs.open(rootdirNEG+"/"+file, 'r',encoding="utf-8") as content_file:
				content = content_file.read() #assume that there are NO "new line characters"
				data.append(content)
	tmpc0=np.zeros(len(data)-len(tmpc1))
	Class=np.concatenate((tmpc1,tmpc0),axis=0)
	return data,Class
#loads unlabelled data	
#returns two lists
#one with the data per file and another with the respective filenames (without the file extension)
def loadUknown(path):
	rootdir=path
	data=[]
	names=[]
	for subdir, dirs, files in os.walk(rootdir):
		for file in files:
			with open(rootdir+"/"+file, 'r', endoding= "utf-8") as content_file:
				content = content_file.read() #assume that there are NO "new line characters"
				data.append(content)
				names.append(file.split(".")[0])
	return data,names

In [2]:
reviews, Class = loadLabeled(path_igor)

## STEPS
- In the train and test set, go from the 2.000.000 movie list to a X movie list - only the movies that are quoted
- To each movie, give a "grade" or a P(positive) and P(negative) to create a new feature
- Create the new feature

In [3]:
# THE MOVIE LIST HAS TO BE COMPLETED
from imdbpie import Imdb
imdb = Imdb(anonymize = True)

# Movie list creation
movie_list = {}
for movie in imdb.top_250():
    movie_list[movie['title']] = movie['rating']

In [ ]:
rev_movie = []
wrong_titles = ['M', 'Up', 'Ran']

for idr,review in enumerate(reviews):   
    # Movie in the review
    movies = []
    for key, value in movie_list.items():
        if key in review and key not in wrong_titles:
            movies.append(value)
    rev_movie.append(movies)

The Matrix  => The worst movie I have seen since Tera Jadoo Chal Gaya. There is no story, no humor, no nothing! The action sequences seem more like a series of haphazard Akshay Kumar Thumbs-Up advertisements stitched together. Heavily influenced from The Matrix and Kung-Fu Hustle but very poorly executed.<br /><br />I did not go a lot of expectations, but watching this movie is an exasperating experience which makes you wonder "What were these guys thinking??!!".<br /><br />The only thing you might remember after watching it is an anorexic Kareena in a bikini.<br /><br />The reason why I did not give a rating of '1' is that every time I think I have seen the worst, Bollywood proves me wrong.
--------------
Sin City  => I had no idea what the film is about before I saw it because Tashan only had teaser trailers while it was being promoted. So I asked my friends if they knew anything about it and they said that "It is the directorial debut of Vijay Krishna Acharya who wrote the screenpla

In [8]:
# TO BE DISCUSSED : WHAT IF MORE THAN 1 MOVIE MENTIONNED?
new_rev_movie = []

for movie in rev_movie:
    # No movies quoted or more than 1 movie quoted
    if len(movie) != 1:
        new_rev_movie.append(-5.4321)
    
    # Only one movie quoted
    else:
        new_rev_movie.append(movie[0])     

In [9]:
# AS FOR NOW, THERE ARE ONLY BAD MOVIES
good_movie_mentionned = []
bad_movie_mentionned = []

for movie in new_rev_movie:
    if movie > 6:
        good_movie_mentionned.append(True)
        bad_movie_mentionned.append(False)
    elif movie < 5:
        good_movie_mentionned.append(False)
        bad_movie_mentionned.append(True)
    else:
        good_movie_mentionned.append(False)
        bad_movie_mentionned.append(False)

In [ ]:
well_classified = 0
bad_classified = 0
for idx, grade in enumerate(good_movie_mentionned):
    if grade == 1 and Class[idx] == 1:
        well_classified += 1
    if grade == 1 and Class[idx] == 0:
        bad_classified += 1

In [ ]:
print float(well_classified)/float((well_classified+bad_classified))

In [10]:
def save_csv(filename, array):
    with open(filename, 'wb') as csvfile:
        writer = csv.writer(csvfile, delimiter = '\n')
        writer.writerow(array)

In [ ]:
save_csv()

# Check if the grades got in the reviews are usefull

In [5]:
rev_grade = []
# Grade/Mark in the review
for idr,review in enumerate(reviews):
    rev_grade.append([])
    review_split= review.split(" ")

    for idw, word in enumerate(review_split):
        for idx, char in enumerate(word):
                if char == '/':
                    ten_is_there= False
                    if(idx < len(word)-2):
                        if word[idx+1] == '1' and word[idx+2] == '0':
                            ten_is_there=True
                    if(idx== len(word) -1 and idw<len(review_split)-1 and len(review_split[idw+1])>1 ):
                        if(review_split[idw+1][0]=='1' and review_split[idw+1][1]=='0'):
                            ten_is_there=True
                    if(ten_is_there):                  
                        if(idx)>0:
                            rev_grade[idr].append(word[0:idx])
                        else:
                            rev_grade[idr].append(review_split[idw-1])

In [6]:
# DO NOT DELETE THE PRINT NOW : The function may be better when used on the test set

# SOMETIMES : 1/10/2015 -> It is a date ! 

def convert_to_real_grade(grade):
    new_grade = 5.4321
    
    ### The grade is a float
    try:
        new_grade = float(grade)
        return float(new_grade)
    
    ### The grade is not a float
    except:
        good = '0123465789'
        numerical_words = {'zero':0, 'one':1, 'two':1, 'three':3, 'four':4, 'five':5, 
                           'six':6, 'seven':7, 'height':8, 'nine':9, 'ten':10}
        
        ## The grade has numerical values at the end 
        if grade[-1] in good:

            ### Read the grade in the string
            one_dot = False
            g_new = ''
            for char in reversed(grade):
                if char in good:
                    g_new = char + g_new
                elif char in '.,' and one_dot == False:
                    g_new  = '.' + g_new
                    one_dot = True
                else:
                    if g_new[0] in '.,':
                        new_grade = g_new[1:]
                    else:
                        new_grade = g_new
                    
        elif (grade[-1] not in good) and (grade.lower() in numerical_words):
            new_grade = numerical_words[grade.lower()]
            
        return float(new_grade)
                
                    

# functions for reviews with more than one grade
def convert_to_real_grade_2(grade):
    final_grade = 5.4321
    new_grades = []
    for i in range(len(grade)):
        new_grade = convert_to_real_grade(grade[i])
        new_grades.append(new_grade)
        
    if final_grade not in new_grades:
        ## NEXT CONDITION IS TO BE DISCUSSED
        if np.max(new_grades) - np.min(new_grades) < 7:
            final_grade = np.mean(new_grades)
    return final_grade

In [13]:
new_rev_grade = []
for idg, grade in enumerate(rev_grade):    
    converted_grade = 5.4321
    
    if grade != []:
        if len(grade) == 1:
            converted_grade = convert_to_real_grade(grade[0])
        else:
            converted_grade = convert_to_real_grade_2(grade)
    new_rev_grade.append(converted_grade)

In [82]:
#Dummies
def get_gummies(new_rev_grade, good_mark, bad_mark):
    good_grade = []
    bad_grade = []

    for grade in new_rev_grade:
        if grade > good_mark and grade <= 10:
            good_grade.append(True)
            bad_grade.append(False)
        elif grade <bad_mark and grade >= 0:
            good_grade.append(False)
            bad_grade.append(True)
        else:
            good_grade.append(False)
            bad_grade.append(False)
            
    return good_grade, bad_grade

In [83]:
def find_best_param(good_grade, bad_grade, Class):
    true_positive = 0
    false_positive = 0
    true_negative = 0
    false_negative = 0
    for idx, good in enumerate(good_grade):
        # The review mentions a good grade
        if good == True and Class[idx] == 1:
            true_positive += 1
        elif good == True and Class[idx] == 0:
            false_positive += 1

        #The review mentions a bad grade
        elif bad_grade[idx] == True and Class[idx] == 0:
            true_negative += 1
        elif bad_grade[idx] == True and Class[idx] == 1:
            false_negative += 1
    
    print "True Positive : ", true_positive, '   -   False Positive : ', false_positive
    print "True Positive / True and false Positive : ", float(true_positive)/float(true_positive+false_positive)
    print "True Negative : ", true_negative, '   -   False Negative', false_negative
    print 'True Negative / True and false negative : ', float(true_negative)/float(true_negative+false_negative)

In [84]:
find_best_param(good_grade, bad_grade, Class)

True Positive :  819    -   False Positive :  26
True Positive / True and false Positive :  0.969230769231
True Negative :  615    -   False Negative 5
True Negative / True and false negative :  0.991935483871


In [85]:
bad_marks = np.linspace(4,4.8,9)
good_marks = np.linspace(6.4,7.2,9)

In [86]:
for good_mark in good_marks:
    for bad_mark in bad_marks:
        good_grades, bad_grades = get_gummies(new_rev_grade, good_mark, bad_mark)
        print 'Good : ', good_mark, '   -   Bad : ', bad_mark
        find_best_param(good_grades, bad_grades, Class)
        print '-------------'

Good :  6.4    -   Bad :  4.0
True Positive :  806    -   False Positive :  21
True Positive / True and false Positive :  0.974607013301
True Negative :  449    -   False Negative 3
True Negative / True and false negative :  0.993362831858
-------------
Good :  6.4    -   Bad :  4.1
True Positive :  806    -   False Positive :  21
True Positive / True and false Positive :  0.974607013301
True Negative :  602    -   False Negative 3
True Negative / True and false negative :  0.995041322314
-------------
Good :  6.4    -   Bad :  4.2
True Positive :  806    -   False Positive :  21
True Positive / True and false Positive :  0.974607013301
True Negative :  602    -   False Negative 4
True Negative / True and false negative :  0.993399339934
-------------
Good :  6.4    -   Bad :  4.3
True Positive :  806    -   False Positive :  21
True Positive / True and false Positive :  0.974607013301
True Negative :  604    -   False Negative 4
True Negative / True and false negative :  0.99342105263

In [91]:
good_grades, bad_grades = get_gummies(new_rev_grade, 6.8, 4.1)
find_best_param(good_grades, bad_grades, Class)

True Positive :  796    -   False Positive :  19
True Positive / True and false Positive :  0.976687116564
True Negative :  602    -   False Negative 3
True Negative / True and false negative :  0.995041322314
